In [1]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

%load_ext autoreload 
%autoreload 2


In [2]:
# TORCH MNIST DISTRIBUTED EXAMPLE

"""run.py:"""
#!/usr/bin/env python
import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp

from pyfig import Pyfig
import numpy as np

n_device = torch.cuda.device_count()
print(n_device)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('cuda: ', device, 'n_dev: ', n_device, 'device', device, 'name: ')

torch.set_num_threads(1)
torch.manual_seed(1234)
torch.set_default_tensor_type(torch.DoubleTensor)   # ❗ Ensure works when default not set AND can go float32 or 64

arg = dict(
    charge = 0,
    spin  = 0,
    a = np.array([[0.0, 0.0, 0.0],]),
    a_z  = np.array([4.,]),
    n_b = 256, 
    n_sv = 32, 
    n_pv = 16, 
    n_corr = 40, 
    n_step = 10000, 
    log_metric_step = 5, 
    exp_name = 'demo',
    # sweep = {},
)

c = Pyfig(wb_mode='disabled', arg=arg, submit=False, run_sweep=False)

n_device = c.n_device
print(f'🤖 {n_device} GPUs available')

: 

In [ ]:
from functorch import make_fx, grad
def f(x):
    return torch.sin(x).sum()
x = torch.randn(100)
grad_f = make_fx(grad(f))(x)
print(grad_f.code)




def forward(self, x_1):
    sin = torch.ops.aten.sin.default(x_1)
    sum_1 = torch.ops.aten.sum.default(sin);  sin = None
    ones_like = torch.ops.aten.ones_like.default(sum_1, dtype = torch.float64, layout = torch.strided, device = device(type='cpu'), pin_memory = False, memory_format = torch.preserve_format);  sum_1 = None
    expand = torch.ops.aten.expand.default(ones_like, [100]);  ones_like = None
    cos = torch.ops.aten.cos.default(x_1);  x_1 = None
    mul = torch.ops.aten.mul.Tensor(expand, cos);  expand = cos = None
    return mul
    


In [ ]:

### model (aka Trainmodel) ### 
from hwat_func import Ansatz_fb
from torch import nn
from hwat_func import compute_ke_b, compute_pe_b
from hwat_func import init_r, get_center_points
import wandb
from hwat_func import keep_around_points, sample_b
from utils import compute_metrix

from functorch import vmap, make_functional, grad, make_fx

_dummy = torch.randn((1,))
dtype = _dummy.dtype
c._convert(device=device, dtype=dtype)
model = c.partial(Ansatz_fb).to(device).to(dtype)

center_points = get_center_points(c.data.n_e, c.app.a)
r = init_r(c.data.n_b, c.data.n_e, center_points, std=0.1)
deltar = torch.tensor([0.02]).to(device).to(dtype)

# from torch.jit import Final    - https://pytorch.org/docs/stable/jit.html type safety and optimisations

# from torch.profiler import profile, record_function, ProfilerActivity

# with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
#     with record_function("model_inference"):
#         model(r[0])

# # print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))
# print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))
print(r.shape)

r.requires_grad_(False)

lp = model(r[0])

mod_com = torch.compile(model)
lp_com = mod_com(r[0])

mod_script = torch.jit.script(model)


Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/energy/amawi/projects/hwat
stdout: 3c51932 stderr: 
Run: ['hostname'] at .
stdout: slid.fysik.dtu.dk  stderr: 
torch.Size([256, 4, 3])


Resource temporarily unavailable (src/thread.cpp:269)


BlockingIOError: [Errno 11] Resource temporarily unavailable

Process ForkProcess-1:
Traceback (most recent call last):
  File "/home/energy/amawi/miniconda3/envs/lumi/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/energy/amawi/miniconda3/envs/lumi/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/energy/amawi/miniconda3/envs/lumi/lib/python3.9/concurrent/futures/process.py", line 240, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/energy/amawi/miniconda3/envs/lumi/lib/python3.9/multiprocessing/queues.py", line 103, in get
    res = self._recv_bytes()
  File "/home/energy/amawi/miniconda3/envs/lumi/lib/python3.9/multiprocessing/connection.py", line 221, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/energy/amawi/miniconda3/envs/lumi/lib/python3.9/multiprocessing/connection.py", line 419, in _recv_bytes
    buf = self._recv(4)
  File "/home/energy/amawi/miniconda3/envs/lumi/lib/pytho

In [ ]:
params

FunctionalModule(
  (stateless_model): RecursiveScriptModule(
    original_name=Ansatz_fb
    (Vs): RecursiveScriptModule(
      original_name=ModuleList
      (0): RecursiveScriptModule(original_name=Linear)
      (1): RecursiveScriptModule(original_name=Linear)
    )
    (Ws): RecursiveScriptModule(
      original_name=ModuleList
      (0): RecursiveScriptModule(original_name=Linear)
      (1): RecursiveScriptModule(original_name=Linear)
    )
    (V_half_u): RecursiveScriptModule(original_name=Linear)
    (V_half_d): RecursiveScriptModule(original_name=Linear)
    (wu): RecursiveScriptModule(original_name=Linear)
    (wd): RecursiveScriptModule(original_name=Linear)
  )
)

In [ ]:

### fix sampler
### fix train step 
### metrix conversion

model_fn, params = make_functional(model)
# model_v = torch.compile(model_fn)
model_v = torch.jit.script(model_fn)
model_v = vmap(model_fn, in_dims=(None, 0))

model_v(params, r)


In [ ]:
opt = torch.optim.RAdam(model.parameters(), lr=0.0001)


wandb.define_metric("*", step_metric="tr/step")
for step in range(1, c.n_step+1):
    
    r, acc, deltar = sample_b(model_v, params, r, deltar, n_corr=c.data.n_corr)  # ❗needs testing 
    r = keep_around_points(r, center_points, l=2.) if step < 1000 else r
    
    model_ke = lambda _r: model_v(params, _r).sum()

    with torch.no_grad():
        ke = compute_ke_b(model_ke, r)
        pe = compute_pe_b(r, c.app.a, c.app.a_z)
        e = pe + ke
        e_mean_dist = torch.mean(torch.abs(torch.median(e) - e))
        e_clip = torch.clip(e, min=e-5*e_mean_dist, max=e+5*e_mean_dist)

    # opt.zero_grad()
    loss_fn = lambda _params: ((e_clip - e_clip.mean())*model_v(_params, r)).mean()
    
    grads = grad(loss_fn)(params)
    for p, g in zip(model.parameters(), grads):
        p.grad = torch.nn.utils.clip_grad_norm_(g.clone(), max_norm=1.)

    opt.step()
    
    params = [p.detach() for p in model.parameters()]
    grads = [p.grad.detach() for p in model.parameters()]
    
    v_tr = dict(
        params=params, grads=grads,
        e=e, pe=pe, ke=ke, r=r,
    )
    
    if not (step % c.log_metric_step):
        metrix = compute_metrix(v_tr)  # ❗ needs converting to torch, ie tree maps
        wandb.log({'tr/step':step, **metrix})